# Settings and utility functions

This module has all of the standard pieces of code we need for the workshop so that we don't have to repeat it for every lesson.

## Set path to CADET bin folder

The first step is to import CADET and tell the system where cadet-cli can be found.

<div class="alert alert-info">

**Note:** 

Please specify the location of the `cadet-cli` executable. It is located in the bin folder where CADET was installed.

</div>

In [ ]:
import platform
from pathlib import Path
from cadet import Cadet

# put in the path to the bin folder
cadet_bin_path = Path.home() / "cadet4.1.0-win10-x64" / "cadet"/ "bin"

if platform.system() == 'Windows':
    cadet_path = cadet_bin_path / "cadet-cli.exe"
    lwe_path = cadet_bin_path / "createLWE.exe"
else:
    cadet_path = cadet_bin_path / "cadet-cli"
    lwe_path = cadet_bin_path / "createLWE"

if cadet_path.exists() and lwe_path.exists():
    Cadet.cadet_path = cadet_path.as_posix()
elif cadet_path.exists() and not lwe_path.exists():
    print("CADET was found but createLWE.exe was not found. Please make sure that none of the files have been moved.")
else:
    print("CADET could not be found. Please check the bin path")

## Standard imports

Here, some other standard libraries are imported.

In [ ]:
import os

from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.display import Image

# python numeric library
import numpy as np

# scientific library for python
import scipy

# pandas is python library for data analysis
import pandas

# addict is a library that makes it easier to create nested dictionaries
from addict import Dict

# json is a standard text based format and it used in CADETMatch for the configuration file
import json

# python plotting library
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='svg'
%matplotlib inline

# jupyter widget support
from ipywidgets import interact, interactive
import ipywidgets as widgets

# Temporary files for simulation objects
import tempfile
tempfile.tempdir = os.path.join(Path.home())

import subprocess

## Template for CADET simulations

This function defines some default values that can be used for all simulations.

In [ ]:
def get_cadet_template(n_units=3):
    cadet_template = Cadet()
    
    cadet_template.root.input.model.nunits = n_units
    
    # Store solution
    cadet_template.root.input['return'].split_components_data = 0
    cadet_template.root.input['return'].split_ports_data = 0
    cadet_template.root.input['return'].unit_000.write_solution_inlet = 1
    cadet_template.root.input['return'].unit_000.write_solution_outlet = 1
    cadet_template.root.input['return'].unit_000.write_solution_bulk = 1
    cadet_template.root.input['return'].unit_000.write_solution_particle = 1
    cadet_template.root.input['return'].unit_000.write_solution_solid = 1
    cadet_template.root.input['return'].unit_000.write_solution_flux = 1
    cadet_template.root.input['return'].unit_000.write_solution_volume = 1
    cadet_template.root.input['return'].unit_000.write_coordinates = 1
    cadet_template.root.input['return'].unit_000.write_sens_outlet = 1
    
    for unit in range(n_units):
        cadet_template.root.input['return']['unit_{0:03d}'.format(unit)] = cadet_template.root.input['return'].unit_000
        
    # Tolerances for the time integrator
    cadet_template.root.input.solver.time_integrator.abstol = 1e-6
    cadet_template.root.input.solver.time_integrator.algtol = 1e-10
    cadet_template.root.input.solver.time_integrator.reltol = 1e-6
    cadet_template.root.input.solver.time_integrator.init_step_size = 1e-6
    cadet_template.root.input.solver.time_integrator.max_steps = 1000000
    
    # Solver settings
    cadet_template.root.input.model.solver.gs_type = 1
    cadet_template.root.input.model.solver.max_krylov = 0
    cadet_template.root.input.model.solver.max_restarts = 10
    cadet_template.root.input.model.solver.schur_safety = 1e-8

    # Run the simulation on single thread
    cadet_template.root.input.solver.nthreads = 1
    
    return cadet_template


def set_discretization(model, nbound=None):
    columns = {'GENERAL_RATE_MODEL', 'LUMPED_RATE_MODEL_WITH_PORES', 'LUMPED_RATE_MODEL_WITHOUT_PORES'}
    
    
    for unit_name, unit in model.root.input.model.items():
        if 'unit_' in unit_name and unit.unit_type in columns:
            unit.discretization.ncol = 20
            unit.discretization.npar = 5
            
            if nbound is None:
                nbound = unit.ncomp*[0]
            unit.discretization.nbound = nbound
            
            unit.discretization.par_disc_type = 'EQUIDISTANT_PAR'
            unit.discretization.use_analytic_jacobian = 1
            unit.discretization.reconstruction = 'WENO'
            unit.discretization.gs_type = 1
            unit.discretization.max_krylov = 0
            unit.discretization.max_restarts = 10
            unit.discretization.schur_safety = 1.0e-8

            unit.discretization.weno.boundary_model = 0
            unit.discretization.weno.weno_eps = 1e-10
            unit.discretization.weno.weno_order = 3

## Function for running CADET

Wrapper for calling the Cadet.run() function with some additional functionality.

In [ ]:
def run_simulation(cadet, file_name=None):
    f = next(tempfile._get_candidate_names())
    file = os.path.join(tempfile.tempdir, f + '.h5')
    
    cadet.filename = file

    # save the simulation
    cadet.save()

    # run the simulation
    data = cadet.run()

    if data.returncode == 0:
        print("Simulation completed successfully")
        cadet.load()   
    else:
        print(data)
        raise Exception("Simulation failed")

    if file_name is not None:
        cadet.filename = file_name

        # save the simulation
        cadet.save()
    else:
        os.remove(file)
            
    return cadet